# **Computational Theory Assessment (2024/2025)**
Ronan Francis (G00403092)
Computational Theory

## **Table of Contents**
1. [Task 1: Binary Representations](#task-1-binary-representations)
2. [Task 2: Hash Functions](#task-2-hash-functions)
3. [Task 3: SHA256 Padding](#task-3-sha256-padding)
4. [Task 4: Prime Numbers](#task-4-prime-numbers)
5. [Task 5: Roots](#task-5-roots)
6. [Task 6: Proof of Work](#task-6-proof-of-work)
7. [Task 7: Turing Machines](#task-7-turing-machines)
8. [Task 8: Computational Complexity](#task-8-computational-complexity)

## **Task 1: Binary Representations**

[Circular Shift](https://en.wikipedia.org/wiki/Circular_shift)

[Bitwise Operators](https://www.geeksforgeeks.org/python-bitwise-operators/)

### 1. [rotl(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotl)
The function `rotl(x, n=1)` that rotates the bits in a 32-bit unsigned integer to the left `n` places.

In [197]:
def rotl(x, n=1):
    # How many bits we need to represent x (at least 1)
    width = x.bit_length() or 1
    mask = (1 << width) - 1 # For example if width is 7, mask is 0b1111111

    # Make sure n is in the range [0, width]
    n %= width

    # Do the rotation:
    # - shift x to the left by n bits
    # - bitwise AND with the mask to keep only the width least significant bits
    # - bitwise OR with the bits that were "shifted out" to the left
    return ((x << n) & mask) | (x >> (width - n))

# Example
x = 0b1010101 # 85
rotated = rotl(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))


Original: 1010101
Rotated:  0101011


### 2. [rotr(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotr)
The function `rotr(x, n=1)` that rotates the bits in a 32-bit unsigned integer to the right `n` places.

In [198]:
def rotr(x, n=1):
    width = x.bit_length() or 1 # How many bits we need to represent x (at least 1)
    mask = (1 << width) - 1  # For example if width is 7, mask is 0b1111111
    x &= mask # Make sure x is in the range [0, 2**width)
    n %= width # Make sure n is in the range [0, width)
    return (x >> n) | ((x << (width - n)) & mask) # Do the rotation

# Example
x = 0b0101010 # 42
rotated = rotl(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))

Original: 101010
Rotated:  010101


### 3. [ch(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)
The function `ch(x, y, z)` that chooses the bits from `y` where `x` has bits set to `1` and bits in `z` where `x` has bits set to `0`.

In [199]:
def ch(x, y, z):
    # (x AND y) XOR (NOT x AND z)
    return (x & y) ^ (~x & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0  
z = 0b1111 # 15

result = ch(x, y, z)
print("Expeted:", bin(y))
print("Result:", bin(result))

Expeted: 0b0
Result: 0b101


### 4. [maj(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)

The function `maj(x, y, z)` which takes a majority vote of the bits in `x`, `y`, and `z`.  
The output should have a `1` in bit position `i` where at least two of `x`, `y`, and `z` have `1`'s in position `i`.  
All other output bit positions should be `0`.

In [200]:
def maj(x, y, z):
    # (x AND y) XOR (x AND z) XOR (y AND z)
    return (x & y) ^ (x & z) ^ (y & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0
z = 0b1111 # 15

print("Expected:", bin(x))
print("Result:  ", bin(maj(x, y, z)))

Expected: 0b1010
Result:   0b1010


[Back to Top](#table-of-contents)

## **Task 2: Hash Functions**

[`Ord()` Function](https://www.w3schools.com/python/ref_func_ord.asp)

[Hash Size](https://www.cimat.mx/ciencia_para_jovenes/bachillerato/libros/%5BKernighan-Ritchie%5DThe_C_Programming_Language.pdf)

In [201]:
def hash(s, base=31, modulus=101):
    hashValue = 0
    for c in s:
        hashValue = ord(c) + base * hashValue # ord(c) returns the ASCII value of c
    return hashValue % modulus 

In [202]:
# Some test strings
test_strings = [
    "hello", "world", "foo", "bar", "baz", "hash", "collision", "test",
    "abcd", "abce", "abcf", "java", "python", "rust", "c++", "javascript"
]

for string in test_strings:
    print(f"{string!r} -> {hash(string)}")

'hello' -> 17
'world' -> 34
'foo' -> 69
'bar' -> 36
'baz' -> 44
'hash' -> 15
'collision' -> 8
'test' -> 86
'abcd' -> 100
'abce' -> 0
'abcf' -> 1
'java' -> 93
'python' -> 91
'rust' -> 17
'c++' -> 60
'javascript' -> 43


*Collison in `rust -> 17` and `hello -> 17`*

In [203]:
import string
import random

def generate_unique_strings(count, length=8):
    """Generate a set of unique random alphanumeric strings of given length."""
    unique_strings = set()
    
    while len(unique_strings) < count:
        new_string = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
        unique_strings.add(new_string)
    
    return list(unique_strings)

def test_collisions(strings, bases, moduli):
    """
    strings:  list of test strings to hash
    bases:    list of possible 'base' values
    moduli:   list of possible 'modulus' values
    
    Returns a dictionary with collision statistics for each (base, modulus) pair.
    """
    
    results = {}
    
    for base in bases:
        for mod in moduli:
            # Dictionary to keep track of hash -> list of strings
            hash_map = {}
            
            for s in strings:
                h = hash(s, base, mod)
                if h not in hash_map:
                    hash_map[h] = [s]
                else:
                    hash_map[h].append(s)
            
            # Count collisions by counting the number of strings in each bucket
            collision_count = sum(len(lst) - 1 for lst in hash_map.values() if len(lst) > 1)
            total_strings = len(strings) 
            
            results[(base, mod)] = {
                "collision_count": collision_count,
                "collision_rate": collision_count / total_strings if total_strings else 0,
                "hash_map": hash_map
            }
    
    return results

In [ ]:
# Example test strings
test_strings = generate_unique_strings(1000, 8)
    
# Possible bases and moduli to test
candidate_bases = [29, 31, 37, 257]
candidate_moduli = [97, 101, 127, 10_007 ] 
    
# Run tests
collision_results = test_collisions(test_strings, candidate_bases, candidate_moduli)
    
# Print summary
for (base, mod), stats in collision_results.items():
    if (base, mod) == (31, 101):
        print("*", end =" ")
    print(f"Base={base}, Modulus={mod} -> Collisions: {stats['collision_count']}, "
        f"Collision Rate: {stats['collision_rate']:.2f}")

Base=29, Modulus=97 -> Collisions: 903, Collision Rate: 0.90
Base=29, Modulus=101 -> Collisions: 899, Collision Rate: 0.90
Base=29, Modulus=127 -> Collisions: 873, Collision Rate: 0.87
Base=29, Modulus=10007 -> Collisions: 62, Collision Rate: 0.06
Base=31, Modulus=97 -> Collisions: 903, Collision Rate: 0.90
* Base=31, Modulus=101 -> Collisions: 899, Collision Rate: 0.90
Base=31, Modulus=127 -> Collisions: 873, Collision Rate: 0.87
Base=31, Modulus=10007 -> Collisions: 47, Collision Rate: 0.05
Base=37, Modulus=97 -> Collisions: 903, Collision Rate: 0.90
Base=37, Modulus=101 -> Collisions: 899, Collision Rate: 0.90
Base=37, Modulus=127 -> Collisions: 873, Collision Rate: 0.87
Base=37, Modulus=10007 -> Collisions: 51, Collision Rate: 0.05
Base=257, Modulus=97 -> Collisions: 903, Collision Rate: 0.90
Base=257, Modulus=101 -> Collisions: 899, Collision Rate: 0.90
Base=257, Modulus=127 -> Collisions: 873, Collision Rate: 0.87
Base=257, Modulus=10007 -> Collisions: 39, Collision Rate: 0.04


### Why `31` and `101`
...

[Back to Top](#table-of-contents)

## **Task 3: SHA256**
...

[Back to Top](#table-of-contents)

## **Task 4: Prime Numbers**
...

[Back to Top](#table-of-contents)

## **Task 5: Roots**
...

[Back to Top](#table-of-contents)

## **Task 6: Proof of Work**
...

[Back to Top](#table-of-contents)

## **Task 7: Turing Machines**
...

[Back to Top](#table-of-contents)

## **Task 8: Computational Complexity**
...

[Back to Top](#table-of-contents)

## **Conclusion**
...

[Back to Top](#table-of-contents)